In [80]:
# !pip install higher tensorboard scipy networkx scikit-learn
import torch
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets,transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import time
import torch.nn.functional as F
import copy
from torch import autograd
import higher
import itertools
import numpy as np
import scipy.sparse as sp
import torch
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
import pickle
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import argparse
import scipy.sparse as sp
import numpy as np
import torch
import ipdb
from scipy.io import loadmat
import networkx as nx
import multiprocessing as mp
import torch.nn.functional as F
from functools import partial
import random
from sklearn.metrics import roc_auc_score, f1_score
from copy import deepcopy
from scipy.spatial.distance import pdist,squareform
import time
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import random
import ipdb
import copy
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.nn import init
import argparse
import scipy.sparse as sp
import numpy as np
import torch
import ipdb
from scipy.io import loadmat
from collections import defaultdict
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# torch.backends.cudnn.enabled = True
# torch.backends.cudnn.benchmark = True

GCN layer based on : https://arxiv.org/abs/1609.02907

In [81]:
class GraphConvolution(Module):
    def __init__(self, in_features, out_features, order, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.order = order
        self.weight = torch.nn.ParameterList([])
        for i in range(self.order):
            self.weight.append(Parameter(torch.FloatTensor(in_features, out_features)))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        for i in range(self.order):
            stdv = 1. / math.sqrt(self.weight[i].size(1))
            self.weight[i].data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj_mtx):
        output = []
        if self.order == 1 and type(adj_mtx) != list:
            adj = [adj_mtx]
        for i in range(self.order):
            support = torch.mm(input, self.weight[i])
            # output.append(support)
            output.append(torch.mm(adj_mtx[i], support))
        output = sum(output)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [ ]:
def my_sigmoid(mx, dim):
    mx = torch.sigmoid(mx)
    return F.normalize(mx, p=1, dim=dim)

Generate the Pima Indian Diabetes (Diabetes) graph through the method used in : https://arxiv.org/abs/2103.00221

In [82]:
diabetes_df = pd.read_csv('./data/diabetes.csv')
# print(diabetes_df.head(10))
G = nx.Graph()
gamma = 4
for patient_id, row_vals in diabetes_df.iterrows():
    G.add_node(str(patient_id), 
    pregnancies = row_vals[0], 
    # glucose = row_vals[1], 
    bloodpressure = row_vals[2], 
    skinthickness = row_vals[3], 
    insulin = row_vals[4], 
    bmi = row_vals[5], 
    diabetespedigreefunction = row_vals[6], 
    age = row_vals[7]) 
    # , outcome = row_vals[8])
# Two loops because of the order problem of NetworkX
for patient_id, row_vals in diabetes_df.iterrows():
    for other_patient_id in range(patient_id + 1, diabetes_df.shape[0]):
        if abs(diabetes_df.iloc[[other_patient_id], 1].values[0] - diabetes_df.iloc[[patient_id], 1].values[0]) < gamma:
            G.add_edge(*(str(patient_id), str(other_patient_id)))
diabetes_adj_mtx = nx.to_numpy_matrix(G)
# print(diabetes_adj_mtx)
# print(G)
# pd.DataFrame(diabetes_adj_mtx).to_csv("data/diabetes_adj_mtx.csv")

Haberman’s survival (Haberman) graph through the method used in : https://arxiv.org/abs/2103.00221

In [88]:
haberman_df = pd.read_csv('./data/haberman.csv')
# print(diabetes_df.head(10))
G = nx.Graph()
gamma = 2
for patient_id, row_vals in haberman_df.iterrows():
    G.add_node(str(patient_id),
        age = row_vals[0],
        operation_year = row_vals[1],
        lymph_nodes = row_vals[2],
    )
    # survival = row_vals[3])
for patient_id, row_vals in haberman_df.iterrows():
    for other_patient_id in range(patient_id + 1, haberman_df.shape[0]):
        if abs(haberman_df.iloc[[other_patient_id], 2].values[0] - haberman_df.iloc[[patient_id], 2].values[0]) < gamma:
            G.add_edge(*(str(patient_id), str(other_patient_id)))
haberman_adj_mtx = nx.to_numpy_matrix(G)
# print(haberman_adj_mtx)
# print(G)
# pd.DataFrame(diabetes_adj_mtx).to_csv("data/haberman_adj_mtx.csv")

Loading the datasets